In [166]:
# Import the QICK drivers and auxiliary libraries
from qick import *
from tqdm.notebook import tqdm
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [27]:
import numpy as np
import matplotlib.pyplot as plt

In [167]:
soc = QickSoc()
soccfg = soc
print(soccfg)


QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v4 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	1:	axis_signal_gen_v4 - tProc output 2, envelope memory 65536 samples
		DAC tile 2, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	2:	axis_signal_gen_v4 - tProc output 3, envelope memory 65536 samples
		DAC tile 2, ch 2, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	3:	axis_signal_gen_v4 - tProc output 4, envelope memory 65536 samples
		DAC tile 2, ch 3, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	4:	axis_signal_gen_v4 - tProc output 5, envelope memory 65536 samples
		DAC tile 3, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	5:	axis_signal_gen_v4 - tProc output 6, envelope memory 65536 samples
		DAC tile 3, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	6:	axis_signal_gen_v4 - tProc out

### Hardware Configuration

generator channel 6   : DAC 229 CH3  <-> Readout channel 0 : ADC 224 CH0

In [168]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        self.declare_gen(ch=cfg["res_ch"], nqz=1)

        self.add_gauss(ch=res_ch, name="measure", sigma=cfg["sigma"], length=cfg["length"])
        maxv=32766
        self.add_pulse(ch=cfg["res_ch"], name="test", idata=np.ones(6400)*maxv)
        
        self.set_pulse_registers(ch=res_ch, outsel="input", style="arb", phase=0, 
                                freq=0, waveform="test", gain=cfg["pulse_gain"], mode="periodic")
        
        #elf.set_pulse_registers(ch=res_ch, outsel="input", style="const", phase=0, 
                                #freq=0, length=, gain=cfg["pulse_gain"])
        
        self.synci(200)
    
    def body(self):
        
        self.pulse(ch=self.cfg["res_ch"])
        self.wait_all()

### Send/receive a pulse with <code> pulse_style </code> = <code> const </code>

In [176]:
config={"res_ch":6,
        "reps":1,
        "relax_delay":1.0,
        "res_phase":0,
        "length":1000,

        "pulse_gain":1000,

        "pulse_freq": 0,

        "adc_trig_offset": 100,

        "soft_avgs":100,
        
        "sigma":128

       }

prog =LoopbackProgram(soccfg, config)
avgq, avgi = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

  0%|          | 0/100 [00:00<?, ?it/s]

In [177]:
soc.reset_gens()

In [59]:
def gen_dc_waveform(gen, b_volt, m_volt=None, freq=None):
    if freq is not None:
        period=1/freq/0.0026
        if period>65535:
            raise Exception('Modulating frequency needs to be higher')
        x_points=np.arange(65535//period*period)
        y_data=m_volt/cfg["gain2dc"][gen]*1000*np.sin(2*np.pi/period*x_points)
    else:
        x_points=np.arange(65535)
        y_data=32767*np.ones(65535)
    y_data+=(b_volt-cfg["dc_offset"][gen])/cfg["gain2dc"][gen]*1000
    plt.plot(x_points, y_data)